In [11]:
import json
import os
from tqdm import tqdm

# 配置路径
JSONL_PATH = "/home/vcj9002/jianshu/workspace/code/ProgressLM/data/raw/visual_demo/visual_h5_franka_3rgb_train.jsonl"
IMAGE_ROOT = "/home/vcj9002/jianshu/workspace/data/robomind/data/images"

missing_images = 0
missing_samples = 0
total_samples = 0

# 可选：保存缺失详情
missing_details = []

with open(JSONL_PATH, "r") as f:
    lines = f.readlines()

for line in tqdm(lines, desc="Checking samples"):
    total_samples += 1
    sample = json.loads(line.strip())
    sample_id = sample["id"]
    image_missing_in_sample = False  # 本样本是否缺失

    # 遍历所有可能包含图片的字段
    for field in ["visual_demo", "stage_to_estimate"]:
        if field in sample and isinstance(sample[field], list):
            for img in sample[field]:
                full_path = os.path.join(IMAGE_ROOT, sample_id, img)
                if not os.path.exists(full_path):
                    missing_images += 1
                    image_missing_in_sample = True
                    missing_details.append(full_path)

    if image_missing_in_sample:
        missing_samples += 1

# ====== 总结统计 ======
print("\n=== 检查完成 ===")
print(f"总样本数          : {total_samples}")
print(f"缺失图像样本数     : {missing_samples}")
print(f"缺失图像总数量     : {missing_images}")


Checking samples:   0%|          | 0/36195 [00:00<?, ?it/s]

Checking samples: 100%|██████████| 36195/36195 [00:01<00:00, 21637.68it/s]


=== 检查完成 ===
总样本数          : 36195
缺失图像样本数     : 3009
缺失图像总数量     : 21879


In [ ]:
import json
import os
from tqdm import tqdm

# === 配置区 === #
JSONL_PATH = "/home/vcj9002/jianshu/workspace/code/ProgressLM/data/raw/visual_demo/visual_h5_franka_3rgb_raw.jsonl"
IMAGE_ROOT = "/home/vcj9002/jianshu/workspace/data/robomind/data/images"
OUTPUT_JSONL = "/home/vcj9002/jianshu/workspace/code/ProgressLM/data/raw/visual_demo/visual_h5_franka_3rgb_train.jsonl"       # 输出的新 jsonl
SAVE_MISSING = False
MISSING_OUTPUT = "missing_images.txt"

# === 统计量 === #
total_samples = 0
kept_samples = 0
removed_samples = 0

missing_details = []  # 记录缺失图片路径（可选）

def iter_images_in_sample(sample):
    """统一提取需要检查的所有图片名称"""
    fields = ["visual_demo", "stage_to_estimate"]
    for field in fields:
        v = sample.get(field, [])
        if isinstance(v, str):
            v = [v]
        if not isinstance(v, list):
            continue
        for x in v:
            x = (x or "").strip()
            if x:
                yield x

# === 读取输入 JSONL === #
with open(JSONL_PATH, "r") as f:
    lines = f.readlines()

# === 输出 JSONL === #
f_out = open(OUTPUT_JSONL, "w")

for line in tqdm(lines, desc="Processing samples", unit="sample"):
    line = line.strip()
    if not line:
        continue

    try:
        sample = json.loads(line)
    except json.JSONDecodeError:
        continue

    total_samples += 1
    sample_id = (sample.get("id") or "").strip()
    if not sample_id:
        # id 缺失，无论如何很难匹配图片，直接丢弃该样本
        removed_samples += 1
        continue

    # 检查该样本是否存在图片缺失
    missing_this_sample = False
    for img_name in iter_images_in_sample(sample):
        full_path = os.path.join(IMAGE_ROOT, sample_id, img_name)
        if not os.path.exists(full_path):
            missing_this_sample = True
            if SAVE_MISSING:
                missing_details.append(full_path)
            # 发现缺失可直接跳出，无需继续检查
            break

    # 如果有缺失 => 丢弃整个样本
    if missing_this_sample:
        removed_samples += 1
    else:
        kept_samples += 1
        f_out.write(json.dumps(sample, ensure_ascii=False) + "\n")

f_out.close()

# === 输出统计信息 === #
print("\n=== 处理完成 ===")
print(f"总样本数           : {total_samples}")
print(f"保留样本数         : {kept_samples}")
print(f"删除样本数         : {removed_samples}")
print(f"输出文件           : {OUTPUT_JSONL}")

if SAVE_MISSING and missing_details:
    with open(MISSING_OUTPUT, "w") as f:
        f.write("\n".join(missing_details))
    print(f"缺失图片详情已保存 : {MISSING_OUTPUT}")

print("\n任务结束 ✅")


Processing samples:   0%|          | 0/36195 [00:00<?, ?sample/s]

Processing samples: 100%|██████████| 36195/36195 [00:00<00:00, 81799.45sample/s]


=== 处理完成 ===
总样本数           : 36195
保留样本数         : 0
删除样本数         : 36195
输出文件           : /home/vcj9002/jianshu/workspace/code/ProgressLM/data/raw/visual_demo/visual_h5_franka_3rgb_train.jsonl

任务结束 ✅


In [ ]:
import json
import os
from pathlib import Path

def process_jsonl_file(input_file, output_file=None):
    """
    处理JSONL文件，删除text_demo中的最后一项[none]
    
    Args:
        input_file: 输入文件路径
        output_file: 输出文件路径，如果为None则覆盖原文件
    """
    # 如果没有指定输出文件，创建临时文件
    if output_file is None:
        output_file = input_file + ".tmp"
        overwrite = True
    else:
        overwrite = False
    
    processed_count = 0
    skipped_count = 0
    error_count = 0
    
    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'w', encoding='utf-8') as outfile:
        
        for line_num, line in enumerate(infile, 1):
            line = line.strip()
            if not line:
                continue
            
            try:
                # 解析JSON
                data = json.loads(line)
                
                # 检查必要的字段是否存在
                if 'text_demo' not in data or 'total_steps' not in data:
                    print(f"警告: 第{line_num}行缺少必要字段，跳过")
                    outfile.write(line + '\n')
                    skipped_count += 1
                    continue
                
                text_demo = data['text_demo']
                total_steps = data['total_steps']
                
                # 检查text_demo是否为列表
                if not isinstance(text_demo, list):
                    print(f"警告: 第{line_num}行text_demo不是列表，跳过")
                    outfile.write(line + '\n')
                    skipped_count += 1
                    continue
                
                # 如果text_demo长度大于total_steps，持续删除最后一个元素
                original_length = len(text_demo)
                removed_count = 0
                
                while len(text_demo) > total_steps:
                    removed_item = text_demo.pop()
                    removed_count += 1
                
                if removed_count > 0:
                    data['text_demo'] = text_demo
                    print(f"✓ 第{line_num}行: 删除了{removed_count}个元素 ({original_length} -> {len(text_demo)}，目标: {total_steps})")
                    processed_count += 1
                elif len(text_demo) == total_steps:
                    print(f"- 第{line_num}行: 长度已匹配 ({len(text_demo)} == {total_steps})，无需处理")
                    skipped_count += 1
                else:
                    # text_demo长度小于total_steps
                    print(f"警告: 第{line_num}行长度不足 ({len(text_demo)} < {total_steps})，无法处理")
                    skipped_count += 1
                
                # 写入处理后的数据
                outfile.write(json.dumps(data, ensure_ascii=False) + '\n')
                
            except json.JSONDecodeError as e:
                print(f"错误: 第{line_num}行JSON解析失败: {e}")
                outfile.write(line + '\n')
                error_count += 1
            except Exception as e:
                print(f"错误: 第{line_num}行处理失败: {e}")
                outfile.write(line + '\n')
                error_count += 1
    
    # 如果是覆盖模式，替换原文件
    if overwrite:
        os.replace(output_file, input_file)
        print(f"\n文件已更新: {input_file}")
    else:
        print(f"\n输出文件: {output_file}")
    
    # 打印统计信息
    print(f"\n处理完成:")
    print(f"  已处理: {processed_count} 行")
    print(f"  已跳过: {skipped_count} 行")
    print(f"  错误: {error_count} 行")


def batch_process_directory(directory, pattern="*.jsonl", output_suffix="_cleaned"):
    """
    批量处理目录下的所有JSONL文件
    
    Args:
        directory: 目录路径
        pattern: 文件匹配模式
        output_suffix: 输出文件后缀（为None则覆盖原文件）
    """
    directory = Path(directory)
    jsonl_files = list(directory.glob(pattern))
    
    if not jsonl_files:
        print(f"未找到匹配 {pattern} 的文件")
        return
    
    print(f"找到 {len(jsonl_files)} 个文件\n")
    
    for file_path in jsonl_files:
        print(f"\n{'='*60}")
        print(f"处理文件: {file_path.name}")
        print('='*60)
        
        if output_suffix:
            # 生成输出文件名
            output_file = file_path.parent / f"{file_path.stem}{output_suffix}{file_path.suffix}"
        else:
            output_file = None
        
        process_jsonl_file(str(file_path), str(output_file) if output_file else None)


if __name__ == "__main__":
    # 使用示例1: 处理单个文件（覆盖原文件）
    process_jsonl_file("/Users/cxqian/Codes/ProgressLM/data/train/text_demo/text_h5_agilex_3rgb_rl.jsonl")
    
    # 使用示例2: 处理单个文件（保存到新文件）
    # process_jsonl_file("input.jsonl", "output.jsonl")
    
    # 使用示例3: 批量处理目录下所有jsonl文件（覆盖原文件）
    # batch_process_directory("./data", pattern="*.jsonl", output_suffix=None)
    
    # 使用示例4: 批量处理目录下所有jsonl文件（保存到新文件）
    # batch_process_directory("./data", pattern="*.jsonl", output_suffix="_cleaned")
    


✓ 第1行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第2行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第3行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第4行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第5行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第6行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第7行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第8行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第9行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第10行: 删除了1个元素 (11 -> 10，目标: 10)
✓ 第11行: 删除了1个元素 (6 -> 5，目标: 5)
✓ 第12行: 删除了1个元素 (6 -> 5，目标: 5)
✓ 第13行: 删除了1个元素 (6 -> 5，目标: 5)
✓ 第14行: 删除了1个元素 (6 -> 5，目标: 5)
✓ 第15行: 删除了1个元素 (6 -> 5，目标: 5)
✓ 第16行: 删除了1个元素 (9 -> 8，目标: 8)
✓ 第17行: 删除了1个元素 (9 -> 8，目标: 8)
✓ 第18行: 删除了1个元素 (9 -> 8，目标: 8)
✓ 第19行: 删除了1个元素 (9 -> 8，目标: 8)
✓ 第20行: 删除了1个元素 (9 -> 8，目标: 8)
✓ 第21行: 删除了1个元素 (9 -> 8，目标: 8)
✓ 第22行: 删除了1个元素 (9 -> 8，目标: 8)
✓ 第23行: 删除了1个元素 (9 -> 8，目标: 8)
✓ 第24行: 删除了1个元素 (10 -> 9，目标: 9)
✓ 第25行: 删除了1个元素 (10 -> 9，目标: 9)
✓ 第26行: 删除了1个元素 (10 -> 9，目标: 9)
✓ 第27行: 删除了1个元素 (10 -> 9，目标: 9)
✓ 第28行: 删除了1个元素 (10 -> 9，目标: 9)
✓ 第29行: 删除了1个元素 (10 -> 9，目标: 9)
✓ 第30行: 删除了1个元素 (10 -> 9，目标: 9)
✓ 第31行: 删除了1个元素 (10 -> 9，目标: 9)
✓ 第32行: 删除